# How do I create custom validators?

Here, we will extend the basic [validation guide](/validate) to enforce more constraints with dictionary-like validators.

This is similar to what is enforced by the CZ CELLxGENE data portal, see [here](https://github.com/chanzuckerberg/single-cell-curation).

## Setup

In [ ]:
!lamin init --storage test-validator --schema bionty

In [ ]:
import lamindb as ln
import lnschema_bionty as lb
from lamin_utils import logger

ln.settings.verbosity = "success"
lb.settings.organism = "human"

## Access data

Let's use an AnnData as the dataset to validate:

In [ ]:
adata = ln.dev.datasets.anndata_human_immune_cells(populate_registries=True)
adata

## Define validation criteria

Define validation criteria for an `AnnData`:

In [ ]:
validators = {
    "var": {"index": lb.Gene.ensembl_gene_id},
    "obs": {
        "donor": ln.ULabel.name,
        "tissue": lb.Tissue.name,
        "cell_type": lb.CellType.name,
        "assay": lb.ExperimentalFactor.name,
    },
}

## Run validation

Run bulk validation:

In [ ]:
features = ln.Feature.lookup()

for slot, slot_validators in validators.items():
    for name, validator in slot_validators.items():
        # access registry (a Django model)
        registry = validator.field.model

        # validate index
        if name == "index":
            logger.print(f"validating {slot}.{name}:")
            index = getattr(adata, slot).index
            validated = registry.validate(index, validator)
            if validated.sum() == len(index):
                logger.success("index matches")

        # validate columns
        else:
            logger.print(f"\nvalidating {slot}.{name}:")
            # check if the column name exist
            if name not in getattr(adata, slot).columns:
                logger.warning(f"{slot}.{name} field is missing")
            else:
                # check if a feature is registered for the column
                if not hasattr(features, name):
                    logger.warning(f"feature '{name}' is not registered")
                # validate categorical labels in a column
                else:
                    labels = getattr(adata, slot)[name]
                    validated = registry.validate(labels, validator)
                    if validated.sum() == len(labels):
                        logger.success("labels match")

Delete test instance:

In [ ]:
!lamin delete --force test-validator